#FastSpeech model research

##Download repository

In [1]:
import os

if not os.path.isdir("/content/FastSpeech2/"):
  !git clone https://github.com/ming024/FastSpeech2

Cloning into 'FastSpeech2'...
remote: Enumerating objects: 657, done.
remote: Total 657 (delta 0), reused 0 (delta 0), pack-reused 657
Receiving objects: 100% (657/657), 290.35 MiB | 24.18 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [2]:
!cd FastSpeech2/ && ls

audio	     hifigan	 model		    README.md	      transformer
config	     img	 prepare_align.py   requirements.txt  utils
dataset.py   index.html  preprocessed_data  synthesize.py
demo	     lexicon	 preprocessor	    text
evaluate.py  LICENSE	 preprocess.py	    train.py


In [3]:
%cd FastSpeech2/

/content/FastSpeech2


##Install requirements

In [4]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 3.1MB 4.3MB/s 
     |████████████████████████████████| 1.6MB 51.2MB/s 
     |████████████████████████████████| 3.6MB 55.7MB/s 
     |████████████████████████████████| 14.6MB 112kB/s 
     |████████████████████████████████| 788kB 51.2MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 645kB 50.7MB/s 
     |████████████████████████████████| 6.8MB 54.5MB/s 
     |████████████████████████████████| 25.9MB 1.3MB/s 
     |████████████████████████████████| 3.0MB 51.9MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 245kB 65.7MB/s 
     |████████████████████████████████| 184kB 58.4MB/s 
     |████████████████████████████████| 20.2MB 312kB/s 
  Created wheel for librosa: filename=librosa-0.7.2-cp37-none-any.whl size=1612903 sha256=31df430198b87e8db8bdafebf71d887cae8cf23b6eea42d7efc120bbd6b1671f
  Stored

###Mounting google disk

In [5]:
import zipfile
import os
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/LJSpeech_900000.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Unpacking pretrained model

In [6]:
!cd tmp && \
 ls && \
mkdir -p ../output/ckpt/LJSpeech/ && \
mv 900000.pth.tar ../output/ckpt/LJSpeech/

900000.pth.tar


In [15]:
!cp "../drive/My Drive/LJSpeech_900000.zip" output/ckpt/LJSpeech/ && \
 ls output/ckpt/LJSpeech/
!unzip "../drive/My Drive/LJSpeech_900000.zip" -d -y output/ckpt/LJSpeech
!ls output/ckpt/LJSpeech

900000.pth.tar	LJSpeech_900000.zip
Archive:  ../drive/My Drive/LJSpeech_900000.zip
caution: filename not matched:  output/ckpt/LJSpeech
900000.pth.tar	LJSpeech_900000.zip


In [7]:
!cd hifigan && unzip generator_LJSpeech.pth.tar.zip && ls

Archive:  generator_LJSpeech.pth.tar.zip
  inflating: generator_LJSpeech.pth.tar  
config.json			generator_universal.pth.tar.zip  models.py
generator_LJSpeech.pth.tar	__init__.py
generator_LJSpeech.pth.tar.zip	LICENSE


In [8]:
!mkdir -p ./output/result/LJSpeech/ && ls ./output/

ckpt  result


In [9]:
!cp /content/drive/MyDrive/output/ckpt/LJSpeech/310000.pth.tar ./output/ckpt/LJSpeech

In [10]:
!ls ./output/ckpt/LJSpeech

310000.pth.tar	900000.pth.tar


###Run pretrained model (Inference)

In [ ]:
import time

# 0.3133268356323242      8
# 0.36463189125061035     13
# 0.36321258544921875     17
# 0.3502969741821289      23
# 0.4021282196044922      29
# 0.42418503761291504     30
# 0.39092564582824707     36
# 0.4430868625640869      51
# 0.48958325386047363     62

lenghts = [8, 13, 17, 23, 29, 30, 36, 51, 62]
times = [0.3133268356323242,
         0.36463189125061035,
         0.3632125854492187,
         0.3502969741821289,
         0.4021282196044922,
         0.42418503761291504,
         0.39092564582824707,
         0.4430868625640869,
         0.48958325386047363]


texts = ["Hello!. ", 
         "How are you? ", 
         "My friend lives. ", 
         "We like to walk in the ", 
         "The fisherman caught a huge. ",
         "Sasha was walking on highway! ",
         "Many guests will come to us for the ",
         "Would you like to walk along the beach and look at ",
         "A flock of cranes flew into the sky, disappearing behind the. "] 

lenghts = [len(seq) for seq in texts]

for i in range(len(texts)):
  seq = texts[i]
  !python3 synthesize.py \
  --text '$seq' \
  --restore_step 310000 \
  --mode single \
  -p config/LJSpeech/preprocess.yaml \
  -m config/LJSpeech/model.yaml \
  -t config/LJSpeech/train.yaml

for i in range(len(lenghts)):
  print(f"{texts[i]} : {lenghts[i]}")


In [32]:
!cd output/result/LJSpeech/ && ls

'Hello world!.png'
'Hello world!_pretrained.png'
'Hello world!_pretrained.wav'
'Hello world!.wav'
'Printing, in the only sense with which we are at present concerned, differs from most if not from al.png'
'Printing, in the only sense with which we are at present concerned, differs from most if not from al.wav'


In [33]:
from IPython.display import Image
Image('Hello world!.png')

In [38]:
# !cp -r ./output/result/LJSpeech/ /content/drive/MyDrive/output/result/LJSpeech/

##Loading dataset

In [5]:
!tar -x -j -f "/content/drive/My Drive/LJSpeech-1.1.tar.bz2" 
!ls

audio	     hifigan	 LJSpeech-1.1	    preprocess.py     train.py
config	     img	 model		    README.md	      transformer
dataset.py   index.html  prepare_align.py   requirements.txt  utils
demo	     lexicon	 preprocessed_data  synthesize.py
evaluate.py  LICENSE	 preprocessor	    text


In [6]:
!mkdir -p /home/ming/Data/ && mv LJSpeech-1.1/ /home/ming/Data/ && ls /home/ming/Data/

LJSpeech-1.1


In [7]:
!unzip ../drive/MyDrive/LJSpeech.zip -d ./preprocessed_data/LJSpeech/TextGrid/ && ls ./preprocessed_data/LJSpeech/TextGrid/

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ005-0284.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ005-0285.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ006-0009.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ006-0014.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ006-0021.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ006-0022.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ006-0031.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ006-0033.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ006-0049.TextGrid  
  inflating: ./preprocessed_data/LJSpeech/TextGrid/TextGrid/LJSpeech/LJ006-0053.TextGrid  
  inflating: ./preproc

##Preprocessing

In [8]:
!python3 prepare_align.py config/LJSpeech/preprocess.yaml

13100it [03:02, 71.81it/s]


In [ ]:
#!wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
#!bash Miniconda3-py39_4.9.2-Linux-x86_64.sh

In [ ]:
#!conda.sh create -n aligner -c conda-forge openblas python=3.7 openfst pynini ngram baumwelch

In [ ]:
#!./montreal-forced-aligner/bin/mfa_train_and_align raw_data/LJSpeech/ lexicon/librispeech-lexicon.txt preprocessed_data/LJSpeech


In [ ]:
# !cat ./preprocessor/preprocessor.py

In [9]:
%%writefile install_mfa.sh
#!/bin/bash

## a script to install Montreal Forced Aligner (MFA)

root_dir=${1:-/tmp/mfa}
mkdir -p $root_dir
cd $root_dir

# download miniconda3
wget -q --show-progress https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p $root_dir/miniconda3 -f

# create py38 env
$root_dir/miniconda3/bin/conda create -n aligner -c conda-forge openblas python=3.8 openfst pynini ngram baumwelch -y
source $root_dir/miniconda3/bin/activate aligner

# install mfa, download kaldi
pip install montreal-forced-aligner # install requirements
pip install git+https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner.git # install latest updates

mfa thirdparty download

echo -e "\n======== DONE =========="
echo -e "\nTo activate MFA, run: source $root_dir/miniconda3/bin/activate aligner"
echo -e "\nTo delete MFA, run: rm -rf $root_dir"
echo -e "\nSee: https://montreal-forced-aligner.readthedocs.io/en/latest/aligning.html to know how to use MFA"

Writing install_mfa.sh


In [ ]:
# download and install mfa
INSTALL_DIR="/tmp/mfa" # path to install directory

!bash ./install_mfa.sh {INSTALL_DIR}
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; mfa align --help

In [ ]:
# install sox tool
!sudo apt install -q -y sox
# convert to 16k audio clips
!mkdir ./wav
!echo "normalize audio clips to sample rate of 16k"
!find raw_data/LJSpeech/ -name "*.wav" -type f -execdir sox --norm=-3 {} -r 16k -c 1 `pwd`/wav/{} \;
!echo "Number of clips" $(ls ./wav/ | wc -l)

In [ ]:
# create transcript files from metadata.csv
lines = open('/home/ming/Data/LJSpeech-1.1/metadata.csv', 'r').readlines()
from tqdm.auto import tqdm
for line in tqdm(lines):
 fn, _, transcript = line.strip().split('|')
 ident = fn
 open(f'./wav/{ident}.txt', 'w').write(transcript)

# this is an example transcript for LJ001-0001.wav
!cat ./wav/LJ001-0001.txt


Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition

In [ ]:
# download a pretrained english acoustic model, and english lexicon
!wget -q --show-progress https://github.com/MontrealCorpusTools/mfa-models/raw/master/acoustic/english.zip
!wget -q --show-progress http://www.openslr.org/resources/11/librispeech-lexicon.txt

english.zip         100%[===================>]  14.05M  35.0MB/s    in 0.4s    
librispeech-lexicon 100%[===================>]   5.37M  2.20MB/s    in 2.4s    


In [ ]:
# FINALLY, align phonemes and speech
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
mfa align -t ./temp -c -j 4 ./wav librispeech-lexicon.txt ./english.zip ./preprocessed_data/LJSpeech

All required kaldi binaries were found!
./temp/wav/align.log
INFO - Setting up corpus information...
INFO - Number of speakers in corpus: 1, average number of utterances per speaker: 13100.0
INFO - Parsing dictionary without pronunciation probabilities without silence probabilities
INFO - Creating dictionary information...
INFO - Setting up training data...
INFO - Generating base features (mfcc)...
INFO - Calculating CMVN...
INFO - Done with setup!
INFO - Performing first-pass alignment...
INFO - Calculating fMLLR for speaker adaptation...
INFO - Performing second-pass alignment...
INFO - All done!


In [ ]:
!mkdir -p ./preprocessed_data/LJSpeech/TextGrid/LJSpeech
!mv ./preprocessed_data/LJSpeech/*.TextGrid ./preprocessed_data/LJSpeech/TextGrid/LJSpeech
!cd ./preprocessed_data/LJSpeech/TextGrid/LJSpeech && rename 's/^....//' *.TextGrid

#!mkdir -p ../drive/MyDrive/my_LJSpeech_preprocessed_data/
#!cp -R ./preprocessed_data/ ../drive/MyDrive/my_LJSpeech_preprocessed_data/

In [10]:
!cp -R ../drive/MyDrive/my_LJSpeech_preprocessed_data/* ./preprocessed_data/

In [11]:
!python3 preprocess.py config/LJSpeech/preprocess.yaml

Processing Data ...
  0% 0/1 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "preprocess.py", line 15, in <module>
    preprocessor.build_from_path()
  File "/content/FastSpeech2/preprocessor/preprocessor.py", line 85, in build_from_path
    if len(pitch) > 0:
UnboundLocalError: local variable 'pitch' referenced before assignment


##Train model 

In [12]:
!python3 train.py -p config/LJSpeech/preprocess.yaml -m config/LJSpeech/model.yaml -t config/LJSpeech/train.yaml --restore_step 310000

Prepare training ...
Traceback (most recent call last):
  File "train.py", line 198, in <module>
    main(args, configs)
  File "train.py", line 41, in main
    model, optimizer = get_model(args, configs, device, train=True)
  File "/content/FastSpeech2/utils/model.py", line 20, in get_model
    ckpt = torch.load(ckpt_path)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 581, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: './output/ckpt/LJSpeech/270000.pth.tar'


In [ ]:
#!cp -R ./preprocessed_data/LJSpeech/ "../drive/MyDrive/my_preprocessed_data"